In [ ]:
#- Load Bronze files with file paths

df_all = spark.read.option("multiline", "true").json("Files/Bronze/") \
    .withColumn("source_file", input_file_name())


In [ ]:
#- Load checkpoint table:

df_checkpoint = spark.table("bronze_file_checkpoints").select("source_file").distinct()

In [ ]:
#- Filter out already-processed files

df_new = df_all.join(df_checkpoint, on="source_file", how="left_anti")

In [ ]:
#- Clean, dedup, and write to silver_user_events

df_deduped.write.mode("append") \
    .option("mergeSchema", "true") \
    .format("delta") \
    .saveAsTable("silver_user_events")


In [ ]:
#- Update checkpoint table:
df_new.select("source_file").distinct() \
    .withColumn("load_time", current_timestamp()) \
    .write.mode("append").format("delta").saveAsTable("bronze_file_checkpoints")